In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import plotly.graph_objects as go


In [4]:
import nltk
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer


In [41]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel
import pyLDAvis.gensim

In [29]:
dataset=pd.read_csv('../input/elon-musks-tweets/data_elonmusk.csv',encoding='latin1')
dataset.head(5)

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk


In [30]:
dataset.drop(['row ID'],axis=1,inplace=True)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet         3218 non-null   object
 1   Time          3218 non-null   object
 2   Retweet from  525 non-null    object
 3   User          3218 non-null   object
dtypes: object(4)
memory usage: 100.7+ KB


In [9]:
#converting the time into datetime
#dataset['Time']=pd.to_datetime(dataset['Time'],format='%y-%m-%d %H:%M:%S')

In [10]:
fig=go.Figure([go.Scatter(x=dataset['Time'])])
fig.show()

In [31]:
dataset['Tweet'][5]

'RT @SpaceX: Supporting the creation of a permanent, self-sustaining human presence on Mars. https://t.co/kCtBLPbSg8 https://t.co/ra6hKsrOcG'

In [32]:
#Text Cleaning

#lowercasing
dataset['Tweet']=dataset['Tweet'].str.lower()
print(dataset['Tweet'][5],'\n')
#removing URl links
dataset['Tweet']=dataset['Tweet'].str.replace('http\S+|www.\S+','',case=False)
print(dataset['Tweet'][5],'\n')
#removing non ascii characters
dataset['Tweet']=dataset['Tweet'].str.encode('ascii','ignore').str.decode('ascii')
print(dataset['Tweet'][5],'\n')
#removing punctations
dataset['Tweet']=dataset['Tweet'].str.replace('[^\w\s]','')
print(dataset['Tweet'][5],'\n')
#removing digits
dataset['Tweet']=dataset['Tweet'].str.replace('\d+','')

#
dataset['Tweet']=dataset['Tweet'].str.replace('rt+','')
print(dataset['Tweet'][5],'\n')
#tokenizing the text
dataset['Tweet']=dataset.apply(lambda x:nltk.word_tokenize(x['Tweet']),axis=1)
print(dataset['Tweet'][5],'\n')
#lemmatizing the words

dataset['Tweet']=dataset['Tweet'].apply(lambda x: (lemmatizer.lemmatize(word)  for word in x))
print(dataset['Tweet'][5],'\n')
#removing stopwords
dataset['Tweet']=dataset['Tweet'].apply(lambda x: ' '.join(word for word in x if word not in (stopwords)))
print(dataset['Tweet'][5],'\n')

dataset['Tweet_dict']=dataset.apply(lambda x:nltk.word_tokenize(x['Tweet']),axis=1)

rt @spacex: supporting the creation of a permanent, self-sustaining human presence on mars. https://t.co/kctblpbsg8 https://t.co/ra6hksrocg 

rt @spacex: supporting the creation of a permanent, self-sustaining human presence on mars.   

rt @spacex: supporting the creation of a permanent, self-sustaining human presence on mars.   

rt spacex supporting the creation of a permanent selfsustaining human presence on mars   

 spacex suppoing the creation of a permanent selfsustaining human presence on mars   

['spacex', 'suppoing', 'the', 'creation', 'of', 'a', 'permanent', 'selfsustaining', 'human', 'presence', 'on', 'mars'] 

<generator object <lambda>.<locals>.<genexpr> at 0x7f7b30ea0750> 

spacex suppoing creation permanent selfsustaining human presence mar 



In [33]:
dataset['Tweet'][:5]

0    meltingice assuming max acceleration g comfoab...
1    spacex bfr capable transpoing satellite orbit ...
2                                           bigajm yup
3                                                   pa
4             fly place eah min anywhere cost per seat
Name: Tweet, dtype: object

In [34]:
#creating dictionary from data
dictionary=corpora.Dictionary(dataset['Tweet_dict'])

In [35]:
doc_term_matrix=[dictionary.doc2bow(tokens) for tokens in dataset['Tweet_dict']]


In [36]:
tfidf=TfidfModel(doc_term_matrix)

In [37]:
corpus_tfidf=tfidf[doc_term_matrix]

In [38]:
corpus_tfidf

In [39]:
topics=10

lda=gensim.models.ldamodel.LdaModel(corpus_tfidf,id2word=dictionary,num_topics=topics)

In [40]:
lda.show_topics()

[(0,
  '0.003*"thanks" + 0.003*"model" + 0.002*"dragon" + 0.002*"california" + 0.002*"time" + 0.002*"yes" + 0.002*"tesla" + 0.002*"car" + 0.002*"course" + 0.002*"review"'),
 (1,
  '0.005*"rocket" + 0.004*"spacex" + 0.003*"falcon" + 0.003*"good" + 0.003*"tesla" + 0.003*"stage" + 0.003*"fire" + 0.003*"landing" + 0.003*"station" + 0.003*"back"'),
 (2,
  '0.003*"model" + 0.003*"tesla" + 0.003*"spacex" + 0.002*"good" + 0.002*"consumer" + 0.002*"really" + 0.002*"attempt" + 0.002*"id_aa_carmack" + 0.002*"rocket" + 0.002*"long"'),
 (3,
  '0.006*"launch" + 0.005*"tesla" + 0.004*"model" + 0.004*"car" + 0.003*"teslamotors" + 0.003*"live" + 0.003*"spacex" + 0.003*"dragon" + 0.003*"solar" + 0.002*"falcon"'),
 (4,
  '0.003*"canaveral" + 0.003*"tesla" + 0.003*"cape" + 0.003*"year" + 0.002*"yes" + 0.002*"new" + 0.002*"falcon" + 0.002*"spacex" + 0.002*"rocket" + 0.002*"wa"')]

In [42]:
pyLDAvis.enable_notebook()
corpora_lda=lda[corpus_tfidf]
topic_display=pyLDAvis.gensim.prepare(lda,corpora_lda,dictionary,mds='tsne')
topic_display

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  33.279987
2       51.163364 -159.968018       2        1  27.960865
3      -73.144554   21.523815       3        1  18.601169
1      -66.410133 -110.606438       4        1  10.088348
0       54.417324   12.443444       5        1  10.069631, topic_info=             Term      Freq     Total Category  logprob  loglift
1079    canaveral  3.000000  3.000000  Default  30.0000  30.0000
68         rocket  6.000000  6.000000  Default  29.0000  29.0000
967       station  2.000000  2.000000  Default  28.0000  28.0000
270        launch  8.000000  8.000000  Default  27.0000  27.0000
1655         fire  2.000000  2.000000  Default  26.0000  26.0000
...           ...       ...       ...      ...      ...      ...
63          first  0.489024  3.646440   Topic5  -6.4962   0.2866
326           one  0.461325  4.046679   Topic5  -6.5545   0.1241
118          good  0.459536  6.449600   Topic5  -6.5584  -0.3459
261   teslamotors  0.456658  6.353965   Topic5  -6.5647  -0.3373
68         rocket  0.434559  6.474364   Topic5  -6.6143  -0.4056

[353 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1647      4  1.190582       abo
2032      3  1.315951    actual
511       2  0.603062   advance
3981      2  1.012085    affect
252       1  0.890202  aificial
...     ...       ...       ...
69        3  0.215915      year
115       1  0.378236       yes
115       2  0.189118       yes
115       4  0.189118       yes
115       5  0.189118       yes

[323 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 2, 1])